Import used packages

In [1]:
import cv2
import numpy as np

Define image path

In [2]:
image_path = './assets/torino.jpg'

Load RGB image

In [3]:
# Load the image in color format
img = cv2.imread(image_path, cv2.IMREAD_COLOR)

Change image to matrix

In [4]:
matrix = np.float32(img)

# Print the matrix shape
print("Matrix shape:", matrix.shape)

Matrix shape: (4000, 6000, 3)


Apply SVD to the matrix